In [8]:
import json
import pandas as pd
import requests
from bs4  import BeautifulSoup

# obtian questions about python from page 1-500

In [1]:
def href(soup):
    # get all href links from one page 
    href=[]
    for i in soup.find_all("a",class_="question-hyperlink",href=True):
        href.append(i['href'])
    return href

In [2]:
def clean_empty_hrefs(hrefs):
   # remove all empty lists
    list_hrefs=[]
    for i in hrefs:
        if i!=[]:
            list_hrefs.append(i)
    # merge all elemenets in one list
    herfs_list=[]
    for i in list_hrefs:
        for j in i:
            herfs_list.append(j)
    return herfs_list

In [3]:
def add_prefix(herfs_list):
    # rearrage those links who do not have 'https://stackoverflow.com' prefix
    new_href=[]
    prefix='https://stackoverflow.com'
    for h in herfs_list:
        if 'https' not in h:
            m=prefix+h+"answertab=votes#tab-top"
            new_href.append(m)
        else:
            new_href.append(h+"answertab=votes#tab-top")
    return new_href

In [4]:
def single_page_scraper(url):
    req=requests.get(url=url)
    soup=BeautifulSoup(req.text,"html.parser")
    return soup

In [5]:
def single_page_question_answer(url):
    page=single_page_scraper(url).find_all("div",class_="post-text",itemprop="text")
    question=[i.find("p").get_text()for i in page][0]
    answer=[i.find("p").get_text() for i in page][1:3]
    
    return question,answer

In [6]:
def questions_answers(start_page,end_page):
    soups=[]
    for page in range(start_page,end_page):
        req=requests.get(url='https://stackoverflow.com/questions/tagged/python?tab=votes&page={}&pagesize=15'.format(page))
        soup=BeautifulSoup(req.text,"html.parser")
        soups.append(soup)
    
    print("Soups are ready!")
    # obtain all href
    hrefs=[]
    for soup in soups:
        hrefs.append(href(soup))
    

    herfs_list=clean_empty_hrefs(hrefs)
    new_hrefs_list=add_prefix(herfs_list)
    print("All hrefs are ready!")

    quesitons=[]
    answers=[]
    for url in new_hrefs_list:
        try:
            q,a=single_page_question_answer(url)
            quesitons.append(q)
            answers.append(a)
        except:
            pass
    print("quesitons and answers are ready!")
    
    new_answers=[]
    for i in range(len(answers)):
        try:
            new_answers.append(answers[i][0])
        except:
            new_answers.append(None)
    print("All most done!")

    new_q = []
    new_a = []
    for i in range(len(new_answers) - 1):
        new_q.append(new_answers[i])
        new_a.append(new_answers[i+1])
    return quesitons+new_q, new_answers+new_a

In [9]:
Answers,Questions=questions_answers(1,500)

Soups are ready!
All hrefs are ready!
quesitons and answers are ready!
All most done!


In [ ]:
Answers1,Questions1=questions_answers(501,1000)

Soups are ready!
All hrefs are ready!


In [ ]:
Answers2,Questions2=questions_answers(1000,2000)

In [ ]:
Answers3,Questions3=questions_answers(2000,3000)

In [ ]:
Answers4,Questions4=questions_answers(3000,4000)

In [76]:
As=Answers+Answers1+Answers2+Answers3+Answers4
Qs=Questions+Questions1+Questions2+Questions3+Questions4

In [77]:
df=pd.DataFrame()
df["question"]=Qs
df["answer"]=As

In [78]:
df.head()

,question,answer
0,What is the use of the yield keyword in Python...,"To understand what yield does, you must unders..."
1,"Given the following code, what does the if __n...",Whenever the Python interpreter reads a source...
2,If Python does not have a ternary conditional ...,"Yes, it was added in version 2.5. The expressi..."
3,"In Python, what are metaclasses and what do we...",A metaclass is the class of a class. A class d...
4,How do you call an external command (as if I'd...,Look at the subprocess module in the standard ...


In [79]:
df.shape

(4717, 2)

In [56]:
df.to_csv("df_python_stackoverflow.csv")